In [1]:
from seai_handler import extract_seai_details
import pandas as pd
import os
#import time
from datetime import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4

#from selenium.webdriver.common.by import By
#from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [2]:
data_dir = 'C:/Users/harry.li/OneDrive - Cushman & Wakefield/realsight/data/'
report_dir = 'C:/Users/harry.li/OneDrive - Cushman & Wakefield/realsight/report/'

In [3]:
options = Options()
#options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation", 'enable-logging'])
#options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.

browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST chromedriver version for 107.0.5304 google-chrome
[WDM] - Driver [C:\Users\Harry\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache
C:\Users\Harry\AppData\Local\Temp\ipykernel_27916\3137345664.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\Harry\AppData\Local\Temp\ipykernel_27916\3137345664.py:8: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [4]:
city_df = pd.DataFrame()
city_df['city'] = [ 'Bristol',
 'Manchester',
#  'Cardiff',
 'Edinburgh',
 'Glasgow',
 'Leeds',
 'Birmingham',
 'Newcastle',
 'London',
]
city_df['max_page'] = [1, 6, 1, 1, 2, 1, 1, 31]

master_df = pd.DataFrame()

In [6]:
for index, row in city_df.iterrows():
    city, max_page = row['city'], row['max_page']
    full_data_list = []
    for page_number in range(1, max_page+1):
        url = "https://buildtorent.io/properties/?address={}&radius=5000&rentMax=30000&sort=RECOMMENDED&page={}&itemsPerPage=20".format(city, page_number)
        browser.get(url)
        browser.implicitly_wait(2)
        data1 = browser.find_element(By.XPATH,'//*[@id="gatsby-focus-wrapper"]/div/main/div/div/div[1]/div/div[3]/div[1]/div/div/div[2]')
        new_data1 = data1.text.split('\nView Properties')
        data2 = browser.find_element(By.XPATH,'//*[@id="gatsby-focus-wrapper"]/div/main/div/div/div[1]/div/div[3]/div[1]/div/div/div[4]')
        new_data2 = data2.text.split('\nView Properties')

        full_data_list = full_data_list + new_data1[:-1] + new_data2[:-1]
        browser.implicitly_wait(0)

    df = pd.DataFrame()
    scheme_name_list = []
    address_list = []
    description_list = []
    onebed_list = []
    twobed_list = []
    threebed_list = []
    studio_list = []
    private_room_list = []
    tag_list = []
    nearest_stations_list = []

    for item in full_data_list:
        info_list = pd.DataFrame(item.split('\n'))
        info_list = info_list[info_list[0]!='']
        info_list.reset_index(inplace=True)

        scheme_name_list.append(info_list[0].loc[0])
        address_list.append(info_list[0].loc[2])
        description_list.append(info_list[0].loc[info_list[0].str.len().idxmax()])
        tag_list.append(list(info_list[0].iloc[3:info_list[0].str.len().idxmax()]))

        col_name = '1-Bedroom'
        index_a = info_list[0][info_list[0].str.contains(col_name)].index
        if info_list[0].iloc[index_a+1].empty:
            onebed_list.append(None)
        else:
            onebed_list.append(info_list[0].iloc[index_a+1].values[0])

        col_name = '2-Bedroom'
        index_a = info_list[0][info_list[0].str.contains(col_name)].index
        if info_list[0].iloc[index_a+1].empty:
            twobed_list.append(None)
        else:
            twobed_list.append(info_list[0].iloc[index_a+1].values[0])

        col_name = '3-Bedroom'
        index_a = info_list[0][info_list[0].str.contains(col_name)].index
        if info_list[0].iloc[index_a+1].empty:
            threebed_list.append(None)
        else:
            threebed_list.append(info_list[0].iloc[index_a+1].values[0])

        col_name = 'Studio'
        index_a = info_list[0][info_list[0].str.contains(col_name)].index
        if info_list[0].iloc[index_a+1].empty:
            studio_list.append(None)
        else:
            studio_list.append(info_list[0].iloc[index_a+1].values[0])

        col_name = 'Private'
        index_a = info_list[0][info_list[0].str.contains(col_name)].index
        if info_list[0].iloc[index_a+1].empty:
            private_room_list.append(None)
        else:
            private_room_list.append(info_list[0].iloc[index_a+1].values[0])
            
        index_a = info_list[0].str.len().idxmax()+ 1
        index_b = info_list[0][info_list[0].str.contains('Bedroom|Studio|Room')].index.min()
        nearest_stations_list.append(list(info_list[0].iloc[index_a:index_b]))
    
    df['Scheme Name'] = scheme_name_list
    df['Address'] = address_list
    df['Tags'] = tag_list
    df['Description'] = description_list
    df['1-Bedroom'] = onebed_list
    df['2-Bedroom'] = twobed_list
    df['3-Bedroom'] = threebed_list
    df['Studio'] = studio_list
    df['Private Room'] = private_room_list
    df['Station Info'] = nearest_stations_list
    df['City'] = city
    master_df = pd.concat([master_df, df])
    print(city)
    
browser.quit()

In [45]:
master_df.columns[4:9]

Index(['1-Bedroom', '2-Bedroom', '3-Bedroom', 'Studio', 'Private Room'], dtype='object')

In [38]:
master_df['Studio'] = master_df['Studio'].str.replace('From ', '')
master_df['Studio'] = master_df['Studio'].str.replace('/month', '')
master_df[master_df.columns[4:9]] = master_df[master_df.columns[4:9]].replace('[\£,]', '', regex=True).astype(float)

In [46]:
report_dir = 'C:/Users/harry/OneDrive - Cushman & Wakefield/realsight/report/'
master_df.to_csv(os.path.join(report_dir, 'BTR_Schemes_top9City.csv'), index=False) 

In [43]:
master_df[master_df.columns[4:9]].replace('[\£,]', '', regex=True).astype(float)

,1-Bedroom,2-Bedroom,3-Bedroom,Studio,Private Room
0,1145.0,1570.0,NaN,905.0,NaN
1,1125.0,1495.0,1845.0,NaN,NaN
2,1160.0,1505.0,2150.0,1040.0,NaN
3,1025.0,1495.0,2250.0,NaN,NaN
4,1250.0,1575.0,NaN,NaN,NaN
...,...,...,...,...,...
611,2383.0,3120.0,3250.0,2166.0,NaN
612,1733.0,NaN,NaN,NaN,NaN
613,2380.0,3044.0,3696.0,1950.0,NaN
614,1625.0,NaN,NaN,NaN,NaN


,Scheme Name,Address,Tags,Description,1-Bedroom,2-Bedroom,3-Bedroom,Studio,Station Info,City
0,Candleriggs Court,"Candleriggs Court, Merchant City, Glasgow, Gla...","[Parking, Bike storage, Controlled access, Smo...",Candleriggs Court offers residents with family...,£750,"£1,250",None,None,"[Argyle Street | 0.2 mi, High Street | 0.2 mi,...",Glasgow
1,Mitchell Apartments,"Kent Road, Glasgow, Glasgow, G3 7BY","[Parcel collection & storage, Bike storage, Li...","Located in Glasgow, Mitchell Apartments is a b...",£895,"£1,195",None,None,"[Charing Cross | 0.1 mi, Anderston | 0.3 mi, S...",Glasgow
2,Westbank Quadrant,"Westbank Quadrant, Glasgow, Glasgow, G12 8NT","[Parking, Controlled access, Common area, Comm...",Westbank Quadrant gives residents the feeling ...,None,"£1,100","£1,795",None,"[Kelvinbridge | 0.2 mi, Hillhead | 0.5 mi, Cha...",Glasgow
3,Devonshire Terrace,"Hyndland, Glasgow, Glasgow, G12 0XE","[Furnished, Parking, Controlled access, Common...",Encompassing high-quality rental flats with am...,£850,"£1,595",None,None,"[Hillhead | 0.4 mi, Hyndland | 0.5 mi, Kelvinh...",Glasgow
4,Hyndland Road,"Hyndland Road, Glasgow, Glasgow, G12 9UY","[Parking, Controlled access, Smoke alarms, Fri...",Encompassing high-quality rental flats with am...,£995,£950,"£1,150",None,"[Hyndland | 0.4 mi, Hillhead | 0.6 mi, Kelvinh...",Glasgow
5,Camphill Avenue,"14 Camphill Avenue, Shawlands, Glasgow, Glasgo...","[Parking, Controlled access, Common area, Comm...","At the heart of Glasgow, Camphill Avenue offer...",None,£695,None,£575,"[Mount Florida | 0.5 mi, Langside | 0.5 mi]",Glasgow


In [97]:
pd.DataFrame(twobed_list).values

array([['£1,250', nan, nan],
       ['£1,195', nan, nan],
       [nan, '£1,100', nan],
       ['£1,595', nan, nan],
       ['£950', nan, nan],
       [nan, nan, '£695']], dtype=object)

In [92]:
twobed_list

[15    £1,250
 Name: 0, dtype: object,
 15    £1,195
 Name: 0, dtype: object,
 13    £1,100
 Name: 0, dtype: object,
 15    £1,595
 Name: 0, dtype: object,
 15    £950
 Name: 0, dtype: object,
 14    £695
 Name: 0, dtype: object]

In [57]:
col_name = '3-Bedroom'
index_a = info_list[0][info_list[0].str.contains(col_name)].index
twobed_list.append(info_list[0].iloc[index_a+1])

In [67]:
pd.to_numeric(info_list[0].iloc[index_a+1])

Series([], Name: 0, dtype: int64)

12

In [116]:
address_list

['261 City Road, Islington, London, EC1V 1LE',
 "2A St Luke's Avenue, Lambeth, London, SW4 7LQ",
 '79a Buckingham Palace Road, Westminster, London, SW1W 0QT',
 'Phillimores Garden, Kensington and Chelsea, London, W8 7HS',
 'Wandsworth Road, Lambeth, London, SW8 2BP',
 'Capitol Way, Brent, London, NW9 0HX',
 '7 Morocco Street, Southwark, London, SE1 3HB',
 'Thunderer Street, Newham, London, E13 9DP']

In [114]:
description_list

['The Lexicon offers contemporary apartments in East Central London that are within walking distance from different amenities and places.',
 'Located in South West London, The Library Building is a Build to Rent that promises safety, support and additional benefits for tenants.',
 'Located at 79a Buckingham Palace Road, South West London, The Nova Building is a splendid choice for those seeking a convenient lifestyle.',
 'Located in West London, The Phillimores is a Build to Rent that caters to growth, connection and a fulfilling lifestyle.',
 'This Space is a Build to Rent in South West London that offers apartments with easy access to transport links and an abundance of activities within walking distance.',
 'Located in North West London, TNQ is a Build to Rent that has generous living areas, beneficial amenities and great access to public transport.',
 'Tower Bridge Gardens offers contemporary apartments that are within walking distance from different amenities and places.',
 'Locat

In [144]:
# btr_name = browser.find_elements(By.TAG_NAME, 'h1') 
# btr_name_list = [i.text for i in btr_name]

# address = browser.find_elements(By.TAG_NAME, 'h6') 
# address_list = [i.text for i in address]